In [33]:
import json
import pprint
import pymongo
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client['articles']
collection = db['aggr_arts_for_word']

In [34]:

# f = open("sorted.json", "a")
# f.write("[")
# l = collection.find().distinct('_id')
# for i in range(0,len(l)):
#     e = collection.find_one({'_id': l[i]})
#     list_of_art = e['articles']
#     newlist = sorted(list_of_art, key=lambda k: k['word_count'], reverse=True) 
#     y = json.dumps({'_id':l[i], 'articles':newlist})
#     f.write(y)
#     f.write(",")
#     if i%1000==0:
#         print(i)
        
# f.write("]")
# f.close()

In [35]:
import nltk
import re
import numpy
import json
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#nltk.download('stopwords')

In [36]:
def get_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [116]:
stop_words = set(stopwords.words('english')) 
lemmatizer = WordNetLemmatizer()

text = "Copies of William Shakespeare’s first four"
pre_t = text.lower()
pre_t = re.sub('[^A-Za-z0-9]+', ' ', pre_t)

word_tokens = word_tokenize(pre_t)  
word_tags = nltk.pos_tag(word_tokens)
word_tags_size = len(word_tags)

#print(k, word_tags_size,"\n\n")

#i - tokenized word index with POS
pos = []
no_sw_list = []

for i in range(0, word_tags_size):
    if word_tags[i][0] not in stop_words:
        word = lemmatizer.lemmatize(word_tags[i][0], get_pos(word_tags[i][1]))
        no_sw_list.append(word)

no_sw_list

collection_sort = db['sorted_articles_for_word']
collection_map = db['word_article_map']

set_pr = {}
set_all = {}
dict_article_fw_c = {}


for i in range(0,len(no_sw_list)):
    x = collection_sort.find_one({'_id': no_sw_list[i]})
    #pprint.pprint(x)
    list_of_art = x['articles']
    #print(list_of_art)
    #{d['article_id']:d['word_count'] for d in list_of_art}
    dict_article_fw_c[no_sw_list[i]] = {d['article_id']:d['word_count'] for d in list_of_art}
    
#     sett_cur = {d['article_id'] for d in list_of_art}
#     if set_pr:
#         set_all = set_all
    
    
#    collection_map.find_one({'word': no_sw_list[i], 'article_id'})

pr_set = {}


sets = dict_article_fw_c[no_sw_list[0]].keys()

d_res = {}
list_of_res = []

if len(no_sw_list)==1:
    d_res = dict_article_fw_c[no_sw_list[0]]
    list_of_res = list(d_res.keys())[0:5]
    
else:
    for i in range(0,len(no_sw_list)):
        pr_set = sets
        sets = sets & dict_article_fw_c[no_sw_list[i]].keys()
        if not sets:
            sets = pr_set

    ls = list(sets)
    ls = list(sets)
    list_of_sum = []
    new_dict = {}

    for i in ls:
        sum = 0
        for j in dict_article_fw_c:
            sum = sum+dict_article_fw_c[j][i]
        new_dict[i] = sum
    
    list_of_res = list(dict(sorted(new_dict.items(), key=lambda item: item[1], reverse=True)).keys())

for i in range(len(list_of_res)):
    print(i+1,list_of_res[i])
    if i==4:
        break

1 23662
2 166182
3 140384
4 147026
5 151909


In [123]:
collection_map.find_one({'word': 'love', 'article_id':17283})

In [ ]:
collection_map